## Maximum a Posteriori Parameter Inference

In this notebook, we introduce the Maximum a Posteriori (MAP), which extends Maximum Likelihood Estimation (MLE) by inclusion of a prior $p(\theta)$ into the cost function. To include this prior information, we construct a Bayesian posterior with Bayes' theorem given as,

$$
P(\theta|D) = \frac{P(D|\theta)P(\theta)}{P(D)}
$$

where,  
$~$  
$P(\theta|D)$ represents the posterior and can be read as "the probability of the parameters $(\theta)$ given the data $(D)$",  
$P(D|\theta)$ is the probability of the data given the parameters, commonly called the likelihood,  
$P(\theta)$ represents the probability of the parameters commonly called the prior,  
$P(D)$ is the probability of the data and is commonly called the marginal probability.  

However, as the marginal probability is commonly difficult to compute and represents a normalisation constant, in the case of MAP this term is forgone and the proportional posterior is optmised instead. This is given as,

$$
P(\theta|D) \propto P(D|\theta)P(\theta)
$$

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.



To demonstrate the MAP process, we will first need a forward model and data to run parameter inference on. As we are introducing this as a simple example, we will use the PyBaMM forward model with white noise as the reference.

We can simulate the model using the `pybamm.Simulation` class. For this example, we use the default current function for the default ("Chen2020") parameter values (5 A) to generate the voltage data. As the goal is to investigate the MAP method, we will generate a range of observations from the forward model.


In [ ]:
model = pybamm.lithium_ion.SPM()
parameter_values = pybamm.ParameterValues("Chen2020")

n = 6  # Number of time-series trajectories
observations = [
    2**j for j in range(1, n + 1)
]  # Number of observations in each trajectory
solutions = []
for i in observations:
    t_eval = np.linspace(0, 10, i)
    solution = pybamm.Simulation(model, parameter_values=parameter_values).solve(
        t_eval=t_eval
    )
    solutions.append(solution)

print(f"Number of observations in each trajectory: {observations}")

Number of observations in each trajectory: [2, 4, 8, 16, 32, 64]


To make the parameter inference more realistic, we add Gaussian noise with zero mean to the data. While this doesn't truly represent the challenge of parameter inference with experimental data, this does ensure the cost landscape curvature isn't perfect. For a more realistic representation of experimental data, a different noise function could be used.

In [ ]:
sigma = 0.005
corrupt_values = solutions[1]["Voltage [V]"](t_eval) + np.random.normal(
    0, sigma, len(t_eval)
)

The reference trajectory needs to be included in the optimisation task, which is handed within the `Dataset` class. In this situation, this class is composed of the time, current, and the noisy voltage data; however, if we were performing parameter inference from a different measured signal, such as 'Cell Temperature', that would need to be included.

In [ ]:
dataset = pybop.Dataset(
    {
        "Time [s]": solutions[1]["Time [s]"](t_eval),
        "Current function [A]": solutions[1]["Current [A]"](t_eval),
        "Voltage [V]": corrupt_values,
    }
)

## Setting up the problem

Next, we need to select the forward model parameters for inference. The PyBOP parameters class composes as many individual PyBOP parameters as the user wants (whether these parameters can be identified is left out of this example). This class requires the parameter name, which must resolve to a parameter within the `ParameterValues` defined above. Additionally, this class can accept an `initial_value` which will be used by the optimiser, as well as bounds. For this example, we will provide a `prior` to the parameter class, which will be used later by the MAP process.

In [ ]:
parameters = {
    "Negative particle radius [m]": pybop.Parameter(
        prior=pybop.Gaussian(4e-6, 1e-6),
    ),
    "Positive particle radius [m]": pybop.Parameter(
        prior=pybop.Gaussian(5e-6, 1e-6),
    ),
}
true_values = [parameter_values[p] for p in parameters.keys()]
parameter_values.update(parameters)

We use a negative Gaussian log-likelihood (NLL) function. Since we have not provided a `sigma` value to the NLL, this will be estimated from the data. `sigma` is the standard deviation of the measurement noise in the dataset.

In [ ]:
simulator = pybop.pybamm.Simulator(
    model,
    parameter_values=parameter_values,
    protocol=dataset,
)
likelihood = pybop.GaussianLogLikelihoodKnownSigma(dataset, sigma0=sigma)
likelihood_problem = pybop.Problem(simulator, likelihood)
posterior = pybop.LogPosterior(likelihood)
problem = pybop.Problem(simulator, posterior)

Discarding duplicate Negative particle radius [m].
Discarding duplicate Positive particle radius [m].


### Plotting the posterior components

Next, to investigate the individual components of the posterior. The `LogPosterior` class provides attributes of the prior and likelihood. To investigate the contributions of each to the posterior we plot the landscapes across a selected parameter range.

In [ ]:
steps = 8  # Number of discretisation points
bounds = np.asarray([[1e-6, 9e-6], [1e-6, 9e-6]])
# pybop.plot.contour(posterior.prior, bounds=bounds, steps=steps, title="Log Prior")
pybop.plot.contour(
    likelihood_problem, bounds=bounds, steps=steps, title="Log Likelihood"
)
pybop.plot.contour(problem, bounds=bounds, steps=steps, title="Log Posterior");

As expected, the prior represents a two-dimensional Gaussian distribution with a mode at $[4e-6,5e-6]$. The likelihood appears to have a banded shape with ridge of optimal points traversing the higher parameter values. Finally, the posterior forms the combination of the two. This is the benefit of the MAP process, as it allows for previous information to be included in the parameter inference task. Previous knowledge is encapsulated within the prior function and influences the posterior, depending on the magnitude of the likelihood function.

## Identifying the maximum a posteriori values

We identify the parameters using the Maximum a Posterior estimate and the Covariance Matrix Adaptation Evolution Strategy (CMAES).

In [ ]:
options = pybop.PintsOptions(
    verbose=True,
    min_iterations=20,
    max_iterations=50,
)
optim = pybop.CMAES(problem, options=options)
result = optim.run()
print("True values:", true_values)

Next, we can plot the MAP results:

In [ ]:
result.plot_convergence()
result.plot_parameters();

| Iter: 1 | Evals: 6| Best Parameters: [3.93552766e-06 6.34738147e-06] | Best Cost: -23.136562908636947


| Iter: 2 | Evals: 12| Best Parameters: [4.42198012e-06 6.06047322e-06] | Best Cost: 124.01755661513752


| Iter: 3 | Evals: 18| Best Parameters: [5.82371545e-06 5.26189673e-06] | Best Cost: 261.97553048804485


| Iter: 4 | Evals: 24| Best Parameters: [5.82371545e-06 5.26189673e-06] | Best Cost: 261.97553048804485


| Iter: 5 | Evals: 30| Best Parameters: [5.82371545e-06 5.26189673e-06] | Best Cost: 261.97553048804485


...

| Iter: 7 | Evals: 42| Best Parameters: [5.82371545e-06 5.26189673e-06] | Best Cost: 261.97553048804485


| Iter: 8 | Evals: 48| Best Parameters: [5.82371545e-06 5.26189673e-06] | Best Cost: 261.97553048804485


| Iter: 9 | Evals: 54| Best Parameters: [5.82371545e-06 5.26189673e-06] | Best Cost: 261.97553048804485


| Iter: 10 | Evals: 60| Best Parameters: [5.82371545e-06 5.26189673e-06] | Best Cost: 261.97553048804485


OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [3.56231470e-06 4.77438501e-06]
  Optimised parameters: [5.77960586e-06 5.37188509e-06]
  Best cost: 262.01800536841364
  Optimisation time: 30.0996356010437 seconds
  Number of iterations: 20
  Number of evaluations: 121
  Reason for stopping: No significant change for 17 iterations.
True values: [5.86e-06, 5.22e-06]


## Concluding thoughts

This notebook illustrates the process of parameter inference with the Maximum a Posteriori method. This process enables encapsulation of prior knowledge into the optimisation process with influence decay as observations of the system increase.